<a href="https://colab.research.google.com/github/konstantine25b/Neural_Network_ML_Facial_Expression_Recognition_Challenge/blob/main/Facial_Expression_Recognition_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Experiment 7

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.4 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle

In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 88% 251M/285M [00:00<00:00, 773MB/s] 
100% 285M/285M [00:00<00:00, 818MB/s]


In [6]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [7]:
!pip install wandb onnx -Uq

# data

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import wandb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [10]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: konstantine25b (konstantine25b-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
class FER2013Dataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx]['pixels']
        emotion = self.data.iloc[idx]['emotion']

        image = np.array([int(pixel) for pixel in pixels.split()]).reshape(48, 48)
        image = image.astype(np.float32) / 255.0

        if self.transform:
            image = self.transform(image)
        else:
            image = torch.FloatTensor(image).unsqueeze(0)

        return image, emotion

In [14]:
def get_transforms():
    train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Pad(4),
        transforms.RandomCrop(48),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    val_test_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    return train_transform, val_test_transform

In [12]:
def create_train_val_loaders(csv_file, batch_size=64, train_split=0.8):
    train_transform, val_transform = get_transforms()

    full_dataset = FER2013Dataset(csv_file, transform=None)

    train_size = int(train_split * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = random_split(
        full_dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_dataset.dataset.transform = train_transform
    val_dataset.dataset.transform = val_transform

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader

In [13]:
def create_test_loader(csv_file, batch_size=64):
    _, test_transform = get_transforms()

    test_dataset = FER2013Dataset(csv_file, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return test_loader

In [15]:
class BasicResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [16]:
class SimpleFacialExpressionResNet(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.3):
        super(SimpleFacialExpressionResNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)

        self.layer1 = BasicResidualBlock(32, 32, stride=1)
        self.layer2 = BasicResidualBlock(32, 64, stride=2)
        self.layer3 = BasicResidualBlock(64, 128, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(128, num_classes)

        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        return x


In [17]:
def test_overfitting():
    print("Testing simple ResNet architecture with overfitting on small dataset...")

    model = SimpleFacialExpressionResNet(dropout_rate=0.0).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_loader, _ = create_train_val_loaders('train.csv', batch_size=32)

    small_batch = []
    for i, (data, target) in enumerate(train_loader):
        small_batch.extend(list(zip(data, target)))
        if len(small_batch) >= 20:
            break

    small_batch = small_batch[:20]

    for epoch in range(30):
        model.train()
        total_loss = 0
        correct = 0

        for data, target in small_batch:
            data, target = data.unsqueeze(0).to(device), torch.tensor([target]).to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

        accuracy = 100. * correct / len(small_batch)
        avg_loss = total_loss / len(small_batch)

        print(f"Overfit Epoch {epoch+1}/30, Loss: {avg_loss:.4f}, Acc: {accuracy:.2f}%")

        if accuracy >= 95.0:
            print("Simple ResNet can overfit successfully!")
            break

    print("Overfitting test completed.\n")


In [18]:
def compute_loss(loader, model, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

    return total_loss / len(loader), 100. * correct / total

In [19]:
def train_model(config=None):
    with wandb.init(project="Facial_Expression_Recognition_6", config=config):
        config = wandb.config

        model = SimpleFacialExpressionResNet(dropout_rate=config.dropout_rate).to(device)
        train_loader, val_loader = create_train_val_loaders('train.csv', config.batch_size)

        criterion = nn.CrossEntropyLoss(label_smoothing=config.label_smoothing)
        optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=3, verbose=True
        )

        best_val_acc = 0
        patience_counter = 0

        wandb.watch(model, log="gradients", log_freq=100)

        for epoch in range(config.epochs):
            model.train()
            train_loss = 0
            train_correct = 0
            train_total = 0

            progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.epochs} [Train]')

            for batch_idx, (data, target) in enumerate(progress_bar):
                data, target = data.to(device), target.to(device)

                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()

                train_loss += loss.item()
                pred = output.argmax(dim=1)
                train_correct += pred.eq(target).sum().item()
                train_total += target.size(0)

                if batch_idx % 50 == 0:
                    wandb.log({
                        "batch_loss": loss.item(),
                        "learning_rate": optimizer.param_groups[0]['lr'],
                        "epoch": epoch
                    })

                progress_bar.set_postfix({
                    'loss': f'{loss.item():.3f}',
                    'acc': f'{100.*train_correct/train_total:.1f}%'
                })

            train_acc = 100. * train_correct / train_total
            train_loss = train_loss / len(train_loader)

            val_loss, val_acc = compute_loss(val_loader, model, criterion, device)

            scheduler.step(val_loss)

            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc,
                "train_val_gap": train_acc - val_acc
            })

            print(f"Epoch {epoch+1}/{config.epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Gap: {train_acc-val_acc:.2f}%")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0

                model_artifact = wandb.Artifact(
                    name=f"best_model_{wandb.run.name}",
                    type="model",
                    description=f"Best simple ResNet model with validation accuracy: {val_acc:.2f}%"
                )

                model_save_dict = {
                    'model_state_dict': model.state_dict(),
                    'model_config': {
                        'num_classes': 7,
                        'dropout_rate': config.dropout_rate
                    },
                    'training_config': dict(config),
                    'val_accuracy': val_acc,
                    'epoch': epoch,
                    'model_architecture': 'SimpleFacialExpressionResNet'
                }

                torch.save(model_save_dict, "best_model.pth")

                model_artifact.add_file("best_model.pth")
                wandb.log_artifact(model_artifact)

                print(f"New best model saved with validation accuracy: {val_acc:.2f}%")
            else:
                patience_counter += 1

            if patience_counter >= config.patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break

        wandb.log({
            "best_val_accuracy": best_val_acc
        })

        print(f"Training completed. Best validation accuracy: {best_val_acc:.2f}%")

        return best_val_acc


In [20]:
def evaluate_model_on_testset(model_artifact_path, test_csv_file, batch_size=64):
    checkpoint = torch.load(model_artifact_path, map_location=device)

    model_config = checkpoint['model_config']
    model = SimpleFacialExpressionResNet(
        num_classes=model_config['num_classes'],
        dropout_rate=model_config['dropout_rate']
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])

    test_loader = create_test_loader(test_csv_file, batch_size)

    criterion = nn.CrossEntropyLoss()
    test_loss, test_acc = compute_loss(test_loader, model, criterion, device)

    print(f"Test Accuracy: {test_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")

    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    print("\nClassification Report:")
    print(classification_report(all_targets, all_preds, target_names=emotion_labels))

    cm = confusion_matrix(all_targets, all_preds)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_labels, yticklabels=emotion_labels)
    plt.title('Confusion Matrix - Facial Expression Recognition (Simple ResNet)')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

    return test_acc, all_preds, all_targets

In [21]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'best_val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 0.0005,
            'max': 0.005
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'dropout_rate': {
            'distribution': 'uniform',
            'min': 0.3,
            'max': 0.6
        },
        'weight_decay': {
            'distribution': 'log_uniform_values',
            'min': 1e-4,
            'max': 1e-2
        },
        'label_smoothing': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 0.3
        },
        'epochs': {
            'value': 25
        },
        'patience': {
            'value': 4
        }
    }
}

In [23]:
test_overfitting()

Testing simple ResNet architecture with overfitting on small dataset...
Overfit Epoch 1/30, Loss: 1.9199, Acc: 25.00%
Overfit Epoch 2/30, Loss: 1.6423, Acc: 40.00%
Overfit Epoch 3/30, Loss: 1.5754, Acc: 40.00%
Overfit Epoch 4/30, Loss: 1.5327, Acc: 40.00%
Overfit Epoch 5/30, Loss: 1.4881, Acc: 40.00%
Overfit Epoch 6/30, Loss: 1.4460, Acc: 40.00%
Overfit Epoch 7/30, Loss: 1.4089, Acc: 40.00%
Overfit Epoch 8/30, Loss: 1.3815, Acc: 40.00%
Overfit Epoch 9/30, Loss: 1.3236, Acc: 40.00%
Overfit Epoch 10/30, Loss: 1.2618, Acc: 40.00%
Overfit Epoch 11/30, Loss: 1.2136, Acc: 40.00%
Overfit Epoch 12/30, Loss: 1.1741, Acc: 50.00%
Overfit Epoch 13/30, Loss: 1.1043, Acc: 55.00%
Overfit Epoch 14/30, Loss: 1.0402, Acc: 55.00%
Overfit Epoch 15/30, Loss: 0.9781, Acc: 60.00%
Overfit Epoch 16/30, Loss: 0.9043, Acc: 60.00%
Overfit Epoch 17/30, Loss: 0.8446, Acc: 60.00%
Overfit Epoch 18/30, Loss: 0.7780, Acc: 80.00%
Overfit Epoch 19/30, Loss: 0.7111, Acc: 90.00%
Overfit Epoch 20/30, Loss: 0.6520, Acc: 90.0

In [24]:
sweep_id = wandb.sweep(sweep_config, project="Facial_Expression_Recognition_6")

Create sweep with ID: 3ss2257j
Sweep URL: https://wandb.ai/konstantine25b-free-university-of-tbilisi-/Facial_Expression_Recognition_6/sweeps/3ss2257j


In [25]:
print("Running simple ResNet hyperparameter sweep...")
wandb.agent(sweep_id, train_model, count=3)


Running simple ResNet hyperparameter sweep...


wandb: Agent Starting Run: p5yjd0pg with config:
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0.4136772971359127
wandb: 	epochs: 25
wandb: 	label_smoothing: 0.29189418822005997
wandb: 	learning_rate: 0.0013036118162858769
wandb: 	patience: 4
wandb: 	weight_decay: 0.004210023573569834


Epoch 1/25 [Train]: 100%|██████████| 180/180 [00:22<00:00,  7.98it/s, loss=1.895, acc=28.5%]


Epoch 1/25, Train Loss: 1.8522, Train Acc: 28.55%, Val Loss: 1.8008, Val Acc: 33.30%, Gap: -4.75%
New best model saved with validation accuracy: 33.30%


Epoch 2/25 [Train]: 100%|██████████| 180/180 [00:22<00:00,  7.99it/s, loss=1.769, acc=37.2%]


...

Epoch 21/25 [Train]: 100%|██████████| 359/359 [00:26<00:00, 13.52it/s, loss=1.059, acc=87.7%]


Epoch 21/25, Train Loss: 1.0348, Train Acc: 87.71%, Val Loss: 1.5927, Val Acc: 53.73%, Gap: 33.99%


Epoch 22/25 [Train]: 100%|██████████| 359/359 [00:26<00:00, 13.47it/s, loss=1.036, acc=88.8%]


Epoch 22/25, Train Loss: 1.0206, Train Acc: 88.84%, Val Loss: 1.5795, Val Acc: 53.85%, Gap: 34.99%
Early stopping triggered after 22 epochs
Training completed. Best validation accuracy: 54.60%


batch_loss,█████▇▇▆▇▆▅▆▄▅▇▆▅▆▄▅▄▄▄▄▅▃▄▃▃▂▂▁▂▂▂▂▂▁▁▂
best_val_accuracy,▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇▇▇██
learning_rate,█████████████████████████▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁
train_accuracy,▁▂▃▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train_loss,█▇▇▆▆▅▅▅▅▄▄▄▄▃▃▂▂▂▂▁▁▁
train_val_gap,▁▂▁▂▂▂▂▃▂▄▃▅▄▄▅▆▇▇▇███
val_accuracy,▁▂▅▅▆▇▇▇█▆█▆▇██▇▇█▇███
val_loss,▇█▃▃▃▂▂▃▁▃▁▃▂▂▃▂▃▂▃▃▃▃
batch_loss,0.98143
best_val_accuracy,54.5977


In [26]:
api = wandb.Api()
runs = api.runs("konstantine25b-free-university-of-tbilisi-/Facial_Expression_Recognition_6")

best_run = None
best_val_acc = 0

In [27]:
for run in runs:
    if run.state == "finished" and "best_val_accuracy" in run.summary:
        val_acc = run.summary["best_val_accuracy"]
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_run = run


In [28]:
if best_run:
    print(f"\nBest run: {best_run.name}")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"Best hyperparameters: {best_run.config}")

    # Download the artifact
    artifacts = api.artifact(f"konstantine25b-free-university-of-tbilisi-/Facial_Expression_Recognition_6/best_model_{best_run.name}:latest")
    download_path = artifacts.download()

    # Check what was actually downloaded
    print(f"Downloaded to: {download_path}")

    # Find the correct path to the model file
    import os
    model_file_path = None

    # Check in the download directory
    if os.path.exists(os.path.join(download_path, "best_model.pth")):
        model_file_path = os.path.join(download_path, "best_model.pth")
    # Check in current directory
    elif os.path.exists("best_model.pth"):
        model_file_path = "best_model.pth"
    # Search recursively
    else:
        for root, dirs, files in os.walk("."):
            if "best_model.pth" in files:
                model_file_path = os.path.join(root, "best_model.pth")
                break

    if model_file_path and os.path.exists(model_file_path):
        print(f"Found model file at: {model_file_path}")

        # Create final artifact with correct path
        final_artifact = wandb.Artifact(
            name="final_best_resnet_model",
            type="model",
            description=f"Final best ResNet model with {best_val_acc:.2f}% validation accuracy"
        )
        final_artifact.add_file(model_file_path)

        # Upload the final artifact
        with wandb.init(project="Facial_Expression_Recognition_6", name="final_model_upload"):
            wandb.log_artifact(final_artifact)
            wandb.log({
                "final_best_val_accuracy": best_val_acc,
                "model_architecture": "ResNet",
                "ready_for_testing": True
            })

        print(f"\nFinal model uploaded to wandb as 'final_best_resnet_model'")
        print("You can now load this model in the future for testing on any dataset!")

        print("\nTo test on a different dataset in the future, use:")
        print("evaluate_model_on_testset('path_to_downloaded_model.pth', 'your_test_dataset.csv')")

    else:
        print("Error: Could not find the downloaded model file!")
        print("Files in current directory:")
        for item in os.listdir("."):
            print(f"  {item}")

else:
    print("No successful runs found!")


Best run: wise-sweep-2
Best validation accuracy: 58.85%
Best hyperparameters: {'epochs': 25, 'patience': 4, 'batch_size': 64, 'dropout_rate': 0.36102178971367016, 'weight_decay': 0.0022730632370258973, 'learning_rate': 0.00408643259183888, 'label_smoothing': 0.2236615416461067}


wandb:   1 of 1 files downloaded.  


Downloaded to: /content/artifacts/best_model_wise-sweep-2:v7
Found model file at: /content/artifacts/best_model_wise-sweep-2:v7/best_model.pth


final_best_val_accuracy,▁
final_best_val_accuracy,58.84709
model_architecture,ResNet
ready_for_testing,True



Final model uploaded to wandb as 'final_best_resnet_model'
You can now load this model in the future for testing on any dataset!

To test on a different dataset in the future, use:
evaluate_model_on_testset('path_to_downloaded_model.pth', 'your_test_dataset.csv')
